## <center> Hofstadter Butterfly </center>##

In [1]:
import plotly.graph_objs as go
import numpy as np
from numpy import pi

In [2]:
def Gear(n): 
    ''' Generates the  Gear-type matrix, i.e. the periodic Jacobi matrix G=(0,..0;1,...1; +1)
    C.W. Gear, A simple set of test matrices for eigenvalue programs,
     Math. Comp. 23, 119-125.
     '''
    G=np.diag(np.ones(n - 1), -1) + np.diag(np.ones(n - 1), 1)
    
    G[0][n-1]=1
    G[n-1][0]=1
    return G


def eigs_Harper(p, q):
   
    d=[2*np.cos(2*np.pi*k*p/q) for k in range(q)] #Harper matrix diagonal        
    Hd=np.diag(d)
    G = Gear(q)
    
    return np.linalg.eigvalsh(Hd+G)#eigenvalues of the Harper matrix 

def gcd(a, b): # Greatest Common Divisor
    if b == 0: return a
    return gcd(b, a % b)

In [3]:
qmax=100

x=[]
y=[]
text=[]
for q in range(4, qmax, 2):
    for p in range(1, q, 2):
        if gcd(p, q) == 1:
            x.extend([p/q]*q) 
            eigs_pq=eigs_Harper(p, q).tolist() 
            y.extend(eigs_pq)           
            p_text=[f"(p, q) = {(p,q)}"]*q
            text.extend([f"{t}<br>E = {round(e, 3)}" for t, e in zip(p_text, eigs_pq)])


In [4]:
data=[dict(type='scatter',
           x=x,
           y=y,
           mode='markers',
           text=text,  
           marker=dict(color='blue', size=1),
           hoverinfo='text')]


In [5]:
axis_style=dict(showline=True, 
                mirror=True, 
                zeroline=False, 
                showgrid=False, 
                ticklen=4)

In [6]:
layout=dict(title='Hofstadter butterfly',
            font=dict(family='Balto'),
            width=600, height=675,
            autosize=False,
            xaxis=dict(axis_style, **dict( title='Phi (magnetic flux)', dtick=0.25)),
            yaxis=dict(axis_style, **dict( title='E (Energy)')),
            hovermode='closest')

In [7]:
fw=go.FigureWidget(data=data, layout=layout)

In [8]:
import plotly.plotly as py

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
py.sign_in('empet', '')
py.iplot(fw, filename='Hofstadter1')

The draw time for this plot will be slow for clients without much RAM.


The Hofstadter butterfly above consists in   N=len(x)*len(y) points, i.e.

In [15]:
len(x)*len(y)

4334378896

In [16]:
from  IPython.core.display  import HTML
def  css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()